# 导入模块

1、机审通过免审核的订单

2、身份证已上传；

3、蚁盾分<30分

4、risk.grade自有评级SABCDEF ;

5、rentOnlineRiskLevelOverall字段，db_credit.risk_alipay_auth_query芝麻租物风控专业版

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

# 定义数据库连接

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
# df_order = pd.read_excel('E:/myfile/p站数据/当月数据/722-804.xlsx')
# df_order.shape
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.auto_pay_tag
from db_digua_business.t_order  om 
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
left join db_digua_business.t_order_stages tos on om.id=tos.order_id
 -- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静')
# and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
-- and om.merchant_id not in (15,18,21,30,31,33,79,85,99)  -- 剔除商家
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -7 day )              -- 近15天数据
-- AND om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -1 day )               -- 近15天数据
;
'''
df_order = query(sql1)
df_order.shape

(343522, 29)

# 合并数据

In [5]:
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(343522)

## 日期处理

In [6]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month

## 备注信息处理 及关键分类 自段处理

In [7]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()

拒绝理由
                                                                 163116
冰鉴火眸分<500命中强拒                                                     54744
命中强拒加入临时黑名单                                                       23316
年龄超过49岁或低于18岁                                                     11052
命中自有模型强拒                                                           4980
                                                                  ...  
命中融安分<700, 命中融安分低于680强拒(芝麻综合风险等级0豁免), 命中青云分<500, 评分等级低于D,命中拒绝        12
综合风险等级为3, 评分等级低于D,命中拒绝                                               12
命中策略240703_14强拒, 命中自有模型强拒, 命中融安分<700, 履约历史等级1, 履约历史等级=1强拒            12
为法院被执行人且未全部结案, 命中云商分低于496强拒, 命中自有模型强拒                                12
命中云商分低于496强拒, 命中自有模型强拒, 蚁盾分>=80                                      12
Name: count, Length: 417, dtype: int64

In [8]:
df = df[df['sku_attributes'].notnull()]

In [9]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")


In [10]:
def xinghao_classify(a,b):
    if 'iphone15promax' in a:
        return 'iphone15promax'
    elif 'iphone14promax' in a:
        return 'iphone14promax'
    elif 'iphone15pro' in a:
        return 'iphone15pro'    
    elif 'iphone14pro' in a:
        return 'iphone14pro' 
    elif 'iphone15plus' in a:
        return 'iphone15plus'  
    elif 'iphone15' in a:
        return 'iphone15' 
    elif 'iphone14' in a:
        return 'iphone14'
    elif 'mate60pro' in a:
        return 'mate60pro'

    elif 'iphone14plus' in a:
        return 'iphone14plus'
    elif 'iphone13promax' in a:
        return 'iphone13promax'
    elif 'mate60' in a:
        return 'mate60'
    elif 'watch' in a:
        return 'watch'
    elif 'iphone13pro' in a:
        return 'iphone13pro'
    elif 'vivox100pro' in a:
        return 'vivox100pro'
    elif '小米13pro' in a:
        return '小米13pro'
    elif '小米13' in a:
        return '小米13'
    elif 'iphone12promax' in a:
        return 'iphone12promax'
    elif 'iphone13' in a:
        return 'iphone13'
    elif '小米14pro' in a:
        return '小米14pro'
    elif '小米14' in a:
        return '小米14'
    elif 'iphone12pro' in a:
        return 'iphone12pro'
    elif 'ipad' in a:
        return 'ipad'
    elif 'iphone12' in a:
        return 'iphone12'
    elif '相机' in a:
        return '相机'
    elif '荣耀x50' in a:
        return '荣耀x50'
    elif 'findx6pro' in a:
        return 'findx6pro'
    elif 'xiaomi13' in a:
        return '小米13'
    elif 'vivoiqoo12pro' in a:
        return 'vivoiqoo12pro'
    elif 'oppofindn2' in a:
        return 'oppofindn2'
    elif 'redminote12turbo' in a:
        return 'redminote12turbo'
    elif 'iqoo11' in a:
        return 'iqoo11'
    elif 'vivox100' in a:
        return 'vivox100'
    elif 'mate50pro' in a:
        return 'mate50pro'
    elif 'x90' in a:
        return 'x90'
    elif 'vivoiqooneo8' in a:
        return 'vivoiqooneo8'
    elif '华为pura70pro' in a:
        return '华为pura70pro'
    elif '华为p60' in a:
        return '华为p60'
    elif 'vivoiqoo12' in a:
        return 'vivoiqoo12'
    elif 'x90pro' in a:
        return 'x90pro'
    else :
        return b

df['型号1'] = df.product_name.str.lower().str.replace(' ','')
# df.型号1.value_counts()

df.loc[:,"型号_new"] = df.apply(lambda x:xinghao_classify(x["型号1"],x['product_name']),axis=1)
# df["型号_new"].value_counts()

## 判断进件

In [11]:
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

进件
未进件    291303
进件      52218
Name: count, dtype: int64

## 判断渠道归属

In [12]:
def qudao_type(a,b):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"]),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                         180759
搜索渠道                          86685
单人聊天会话中的小程序消息卡片（分享）           16548
八派信息                          13332
其他渠道场景渠道。                     11208
生活号                            8472
我的小程序入口                        6474
九州信息                           2976
付费灯火                           2892
小舞直播                           2832
小程序商家消息（服务提醒）                  2124
支付宝社群                          2088
未知渠道                           2016
抖音渠道                           1644
付费流量(通过商家数字推广平台，灯火等投放的广告)      1032
支付宝 push 消息（同1014）              720
支付成功页                           435
苹果SL23                          408
线下易拉宝                           240
扫描二维码                           240
天翌科技                             84
系统桌面图标                           72
卡包\r\n                           48
曙光计划                             48
繁星                               24
数立月活SL22                         24
租iPhone16                        24
扫一扫入口(抖音极速版)           

## 订单预排序去重  剔除内部订单

In [13]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

## 定义审核状态

In [14]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [15]:
def reject_type(a,b,c,d,e,f,g,h,i,j):
    if h==1 and b=="未进件":
        return "前置拦截"
    elif i==1 and b=="进件":
        return "机审强拒"    
    #抖音渠道
    # elif h==1 and b=="进件":
    #     return "机审强拒"  
    elif  b=="进件" and c is not np.nan:
        return "人审拒绝"
    elif  b=="进件" and d is not np.nan:
        return "客户取消"
    elif  b=="进件" and f is not np.nan:
        return "无法联系"
    elif  b=="进件" and j==1 :
        return "出库前风控强拒"
    elif  b=="进件" and "待审核" in e:
        return "待审核"
    elif  b=="进件" and "待发货" in e:
        return "出库"
    elif  b=="进件" and "待收货" in e:
        return "出库"
    elif b=="进件" and "租赁中" in e:
        return "出库"
    elif  b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    elif  b=="未进件":
        return "进件前取消"
    else :
        return e

df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()  

审核状态
进件前取消      11161
前置拦截        4675
机审强拒        2572
出库           581
人审拒绝         325
客户取消         276
已退款          146
出库前风控强拒      121
待审核           77
无法联系          25
待归还            1
Name: count, dtype: int64

## 区分进件数据源

In [16]:
# df=df_insure1

# %%
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j=df[df["进件"]=="进件"]

### 今日日期

In [17]:
Today= datetime.date.today()
str(Today)

'2024-11-11'

In [18]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)
df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)
df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)

# 去除商家数据

In [19]:
#剔除商家数据只保留自营租机业务数据
df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)
df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)  # 趣智数码  单
df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)

In [20]:
df.shape

(18915, 63)

In [21]:
# df.groupby(['下单日期']).agg({'order_id':'size','是否进件':'sum','进件前取消':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})

In [22]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'auto_pay_tag', '下单日期', '月份',
       '拒绝理由', '取消原因', '电审拒绝原因', '无法联系原因', '商品类型', '租赁方案', '押金类型', '优惠券使用否',
       '是否进行预授权', '颜色', '内存', '进件', '来源渠道', '型号1', '型号_new', '归属渠道', '状态编码',
       '是否前置拦截', '是否机审强拒', '是否出库前风控强拒', '审核状态', '下单时段', '待审核', '前置拦截', '机审强拒',
       '人审拒绝', '客户取消', '出库前风控强拒', '无法联系', '是否进件', '是否出库', '进件前取消'],
      dtype='object')

In [23]:
df['create_date'] = pd.to_datetime(df['create_date'])
df['create_time'] = pd.to_datetime(df['create_time'])

In [24]:
df['create_hour'] = df['create_time'].dt.hour

In [25]:
# 机审免审的订单占总订单的比例
def pass_rate_ms(df):
    # 进件数量
    df_jj_pass = df[df['是否进件']==1]
    # 机审通过数量
    df_risk_pass = df[(df['是否进件']==1)&(df['机审强拒']!=1)]
    # 免审数量
    df_ms_pass = df_risk_pass[df_risk_pass['decision_result'].str.contains(pat='免人审',regex=False)==True]
    # 免审比例
    ms_pass_rate = df_ms_pass.shape[0]/df_jj_pass.shape[0]
    ms_pass_rate = np.round(ms_pass_rate,2)

    # 人审比例
    p_pass_rate = (df_risk_pass.shape[0] - df_ms_pass.shape[0])/df_jj_pass.shape[0]
    p_pass_rate =  np.round(p_pass_rate,2)
    # ms_pass_rate = f'{ms_pass_rate:.2}'
    return ms_pass_rate,p_pass_rate



In [26]:
# 有30%左右的客户会进入免人审
ms_pass_rate,p_pass_rate = pass_rate_ms(df)
ms_pass_rate,p_pass_rate

(np.float64(0.25), np.float64(0.09))

In [27]:
df.head()

,create_date,create_time,order_id,order_number,all_money,status,status2,channel_type,sku_attributes,product_name,...,前置拦截,机审强拒,人审拒绝,客户取消,出库前风控强拒,无法联系,是否进件,是否出库,进件前取消,create_hour
0,2024-11-04,2024-11-04 00:00:21,1679429,A202411040000211,4720.26,13,订单取消,芝麻租物,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",首月1元 iPhone16Pro 全新未激活 赠送壳膜 国行正品 顺丰包邮,...,0,0,0,0,0,0,0,0,1,0
60,2024-11-04,2024-11-04 00:00:47,1679434,A202411040000466,4598.11,13,订单取消,芝麻租物,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",首月5折 准新iPhone16 国行正品 赠送壳膜 审核快 寄出包邮,...,1,0,0,0,0,0,0,0,0,0
96,2024-11-04,2024-11-04 00:01:02,1679437,A202411040001029,5039.62,13,订单取消,芝麻租物,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",iPhone15Pro 首月1元 非监管机 全新国行 【芝麻活动专享3】,...,1,0,0,0,0,0,0,0,0,0
144,2024-11-04,2024-11-04 00:02:48,1679441,A2024110400024713,6435.40,13,订单取消,芝麻租物,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",首月1元 iPhone16ProMax 全新未激活 赠送壳膜 国行正品 顺丰包邮,...,1,0,0,0,0,0,0,0,0,0
192,2024-11-04,2024-11-04 00:03:43,1679445,A2024110400034217,8734.43,13,订单取消,芝麻租物,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",首月1元 iPhone16ProMax 全新未激活 赠送壳膜 国行正品 顺丰包邮,...,0,0,0,0,0,0,0,0,1,0


In [28]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'auto_pay_tag', '下单日期', '月份',
       '拒绝理由', '取消原因', '电审拒绝原因', '无法联系原因', '商品类型', '租赁方案', '押金类型', '优惠券使用否',
       '是否进行预授权', '颜色', '内存', '进件', '来源渠道', '型号1', '型号_new', '归属渠道', '状态编码',
       '是否前置拦截', '是否机审强拒', '是否出库前风控强拒', '审核状态', '下单时段', '待审核', '前置拦截', '机审强拒',
       '人审拒绝', '客户取消', '出库前风控强拒', '无法联系', '是否进件', '是否出库', '进件前取消',
       'create_hour'],
      dtype='object')

# 导出数据 去重订单明细  

In [29]:
# df.to_excel(f"E:/myfile/p站数据/订单日报/去重订单明细7月_{Today}.xlsx")
# 

In [30]:
# 按小时进件排查
def group_data_hour(df0):
    df_group = df0.groupby(['create_date','create_hour']).agg({'order_id':'size','是否进件':'sum','进件前取消':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
    df_group.rename(columns={'order_id':'去重订单数'},inplace=True )

    df_group["拦截率"] = df_group["前置拦截"]/df_group["去重订单数"]
    df_group["拦截率"] = df_group["拦截率"].apply(lambda x:format(x,".2%"))

    df_group["进件前取消率"] = df_group["进件前取消"]/df_group["去重订单数"]
    df_group["进件前取消率"] = df_group["进件前取消率"].apply(lambda x:format(x,".2%"))

    df_group['预授权通过率'] = df_group["是否进件"]/df_group["去重订单数"]
    df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x:format(x,".2%"))

    df_group["强拒比例"] = df_group["机审强拒"]/df_group["是否进件"]
    df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["机审通过件"] = df_group["是否进件"]-df_group["机审强拒"]

    df_group["风控通过件"] = df_group["是否进件"]-df_group["机审强拒"]-df_group["人审拒绝"]
    df_group["风控通过率"] = df_group["风控通过件"]/df_group["是否进件"]
    df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

    df_group["取消率"] = df_group["客户取消"]/df_group["是否进件"]
    df_group["取消率"] = df_group["取消率"].apply(lambda x:format(x,".2%"))

    df_group["无法联系占比"]=df_group["无法联系"]/df_group["是否进件"]
    df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

    df_group["人审拒绝率"]=df_group["人审拒绝"]/df_group["是否进件"]
    df_group["人审拒绝率"]=df_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

    df_group["出库前强拒比例"]=df_group["出库前风控强拒"]/df_group["是否进件"]
    df_group["出库前强拒比例"]=df_group["出库前强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["进件出库率"]=df_group["是否出库"]/df_group["是否进件"]
    df_group["进件出库率"]=df_group["进件出库率"].apply(lambda x:format(x,".2%"))

    df_group["订单出库率"]=df_group["是否出库"]/df_group["去重订单数"]
    df_group["订单出库率"]=df_group["订单出库率"].apply(lambda x:format(x,".2%"))

    return df_group

In [31]:
from datetime import datetime
def check_df_er(data,time_list):
    # data_list_2 = ['2024-09-03', '2024-09-04', '2024-09-05', '2024-09-06', '2024-09-07']
    time_list = [datetime.strptime(date, '%Y-%m-%d') for date in time_list] 
    df_er = df[df['create_date'].isin(time_list)]
    df_all_hour = group_data_hour(df_er)
    df_all_hour =df_all_hour[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","人审拒绝率","出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]
    return  df_all_hour

In [32]:
data_list_2 = ['2024-08-28', '2024-08-29', '2024-08-30','2024-08-31','2024-09-01','2024-09-02','2024-09-03','2024-09-04', '2024-09-05', '2024-09-06']
df_er2 = check_df_er(df,data_list_2)
df_er2

,,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,风控通过件,风控通过率,人审拒绝率,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
create_date,create_hour,,,,,,,,,,,,,,,,


In [33]:
# df_er2.to_excel('F:/myfile/p站数据/订单日报/20240907异常排查.xlsx')

In [34]:
# data_list_1 = ['2024-09-04', '2024-09-05', '2024-09-06']
# df_er1 = check_df_er(df,data_list_1)
# df_er1 

# 总体转化数据情况

In [35]:
def group_data(df0):
    df_group = df0.groupby(['下单日期']).agg({'order_id':'size','是否进件':'sum','进件前取消':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
    df_group.rename(columns={'order_id':'去重订单数'},inplace=True )

    df_group["拦截率"] = df_group["前置拦截"]/df_group["去重订单数"]
    df_group["拦截率"] = df_group["拦截率"].apply(lambda x:format(x,".2%"))

    df_group["进件前取消率"] = df_group["进件前取消"]/df_group["去重订单数"]
    df_group["进件前取消率"] = df_group["进件前取消率"].apply(lambda x:format(x,".2%"))

    df_group['预授权通过率'] = df_group["是否进件"]/df_group["去重订单数"]
    df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x:format(x,".2%"))

    df_group["强拒比例"] = df_group["机审强拒"]/df_group["是否进件"]
    df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["机审通过件"] = df_group["是否进件"]-df_group["机审强拒"]

    df_group["风控通过件"] = df_group["是否进件"]-df_group["机审强拒"]-df_group["人审拒绝"]
    df_group["风控通过率"] = df_group["风控通过件"]/df_group["是否进件"]
    df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

    df_group["取消率"] = df_group["客户取消"]/df_group["是否进件"]
    df_group["取消率"] = df_group["取消率"].apply(lambda x:format(x,".2%"))

    df_group["无法联系占比"]=df_group["无法联系"]/df_group["是否进件"]
    df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

    df_group["人审拒绝率"]=df_group["人审拒绝"]/df_group["是否进件"]
    df_group["人审拒绝率"]=df_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

    df_group["出库前强拒比例"]=df_group["出库前风控强拒"]/df_group["是否进件"]
    df_group["出库前强拒比例"]=df_group["出库前强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["进件出库率"]=df_group["是否出库"]/df_group["是否进件"]
    df_group["进件出库率"]=df_group["进件出库率"].apply(lambda x:format(x,".2%"))

    df_group["订单出库率"]=df_group["是否出库"]/df_group["去重订单数"]
    df_group["订单出库率"]=df_group["订单出库率"].apply(lambda x:format(x,".2%"))

    return df_group

In [36]:
#总体
df_all = group_data(df)

df_all[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]  
#df_all#[27:28]#.to_excel("C:/Users/Administrator/Desktop/订单日报/总体数据转化1031.xlsx")


,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,风控通过件,风控通过率,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
下单日期,,,,,,,,,,,,,,,
2024-11-04,2587,618,23.89%,584,22.57%,418,71.58%,166,150,25.68%,23,3.94%,88,15.07%,3.40%
2024-11-05,2541,588,23.14%,577,22.71%,410,71.06%,167,152,26.34%,21,3.64%,84,14.56%,3.31%
2024-11-06,2644,664,25.11%,538,20.35%,364,67.66%,174,138,25.65%,13,2.42%,80,14.87%,3.03%
2024-11-07,2665,641,24.05%,560,21.01%,362,64.64%,198,155,27.68%,17,3.04%,86,15.36%,3.23%
2024-11-08,2574,595,23.12%,541,21.02%,311,57.49%,230,161,29.76%,14,2.59%,80,14.79%,3.11%
2024-11-09,2577,677,26.27%,490,19.01%,331,67.55%,159,135,27.55%,11,2.24%,67,13.67%,2.60%
2024-11-10,2491,641,25.73%,500,20.07%,305,61.00%,195,172,34.40%,17,3.40%,80,16.00%,3.21%
2024-11-11,836,248,29.67%,127,15.19%,71,55.91%,56,39,30.71%,5,3.94%,5,3.94%,0.60%


导出数据  总体转化

In [37]:
# df_all.to_excel('E:/myfile/p站数据/订单日报/日报相关数据输出_{Today}.xlsx')

In [38]:
# 搜索渠道转化情况

## 搜索渠道转化情况

In [39]:
df_ss = group_data(df[df["归属渠道"]=="搜索渠道"]) 
df_ss[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]


,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,风控通过件,风控通过率,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
下单日期,,,,,,,,,,,,,,,
2024-11-04,774,58,7.49%,247,31.91%,201,81.38%,46,43,17.41%,3,1.21%,32,12.96%,4.13%
2024-11-05,776,60,7.73%,234,30.15%,186,79.49%,48,46,19.66%,6,2.56%,31,13.25%,3.99%
2024-11-06,733,72,9.82%,192,26.19%,135,70.31%,57,46,23.96%,2,1.04%,34,17.71%,4.64%
2024-11-07,740,59,7.97%,220,29.73%,146,66.36%,74,58,26.36%,1,0.45%,40,18.18%,5.41%
2024-11-08,813,61,7.50%,242,29.77%,131,54.13%,111,74,30.58%,3,1.24%,41,16.94%,5.04%
2024-11-09,668,69,10.33%,180,26.95%,119,66.11%,61,55,30.56%,4,2.22%,39,21.67%,5.84%
2024-11-10,588,56,9.52%,200,34.01%,126,63.00%,74,66,33.00%,3,1.50%,40,20.00%,6.80%
2024-11-11,140,19,13.57%,34,24.29%,17,50.00%,17,7,20.59%,0,0.00%,0,0.00%,0.00%


数据写入已有表格中新的sheet

In [40]:
# with pd.ExcelWriter('E:/myfile/订单日报数据/日报相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     df_ss.to_excel(writer, sheet_name='搜索渠道转化', index=False)  # index=False 表示不导出行索引  

## 芝麻租物渠道转化

In [41]:
df_zw = group_data(df[df["归属渠道"]=="芝麻租物"]) 
df_zw[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]


,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,风控通过件,风控通过率,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
下单日期,,,,,,,,,,,,,,,
2024-11-04,1237,368,29.75%,231,18.67%,137,59.31%,94,85,36.80%,11,4.76%,50,21.65%,4.04%
2024-11-05,1145,328,28.65%,243,21.22%,164,67.49%,79,74,30.45%,6,2.47%,39,16.05%,3.41%
2024-11-06,1198,368,30.72%,223,18.61%,148,66.37%,75,67,30.04%,8,3.59%,33,14.80%,2.75%
2024-11-07,1238,376,30.37%,239,19.31%,160,66.95%,79,67,28.03%,5,2.09%,36,15.06%,2.91%
2024-11-08,1091,330,30.25%,194,17.78%,118,60.82%,76,62,31.96%,7,3.61%,27,13.92%,2.47%
2024-11-09,1284,416,32.40%,202,15.73%,133,65.84%,69,58,28.71%,3,1.49%,23,11.39%,1.79%
2024-11-10,1403,428,30.51%,223,15.89%,130,58.30%,93,85,38.12%,9,4.04%,34,15.25%,2.42%
2024-11-11,533,183,34.33%,65,12.20%,41,63.08%,24,22,33.85%,3,4.62%,4,6.15%,0.75%


In [42]:
# with pd.ExcelWriter('E:/myfile/订单日报数据/日报相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     df_zm.to_excel(writer, sheet_name='芝麻租物渠道转化', index=False)  # index=False 表示不导出行索引  

## 单人聊天会话中的小程序消息卡片（分享

In [43]:
# df0=df[df["来源渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
# df_dr = group_data(df[df["来源渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]) 
# df_dr[["去重订单数","前置拦截","拦截率","是否进件","进件率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]


In [44]:
# with pd.ExcelWriter('E:/myfile/订单日报数据/日报相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     df_dr.to_excel(writer, sheet_name='单人聊天会话小程序渠道转化', index=False)  # index=False 表示不导出行索引  

## 抖音 转化数据*****

In [45]:

df_dy = group_data(df[df["归属渠道"]=="抖音渠道"]) 
df_dy[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例"
       ,"是否出库","进件出库率","订单出库率",'待审核','人审拒绝','客户取消']]


,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,风控通过件,风控通过率,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率,待审核,人审拒绝,客户取消
下单日期,,,,,,,,,,,,,,,,,,
2024-11-04,20,5,25.00%,3,15.00%,2,66.67%,1,1,33.33%,0,0.00%,1,33.33%,5.00%,0,0,0
2024-11-05,25,5,20.00%,11,44.00%,5,45.45%,6,3,27.27%,2,18.18%,0,0.00%,0.00%,0,3,1
2024-11-06,10,1,10.00%,3,30.00%,3,100.00%,0,0,0.00%,0,0.00%,0,0.00%,0.00%,0,0,0
2024-11-07,3,1,33.33%,1,33.33%,0,0.00%,1,0,0.00%,0,0.00%,0,0.00%,0.00%,0,1,0
2024-11-08,13,0,0.00%,5,38.46%,4,80.00%,1,1,20.00%,0,0.00%,0,0.00%,0.00%,0,0,0
2024-11-09,1,0,0.00%,1,100.00%,1,100.00%,0,0,0.00%,0,0.00%,0,0.00%,0.00%,0,0,0
2024-11-10,1,0,0.00%,0,0.00%,0,nan%,0,0,nan%,0,nan%,0,nan%,0.00%,0,0,0
2024-11-11,1,0,0.00%,0,0.00%,0,nan%,0,0,nan%,0,nan%,0,nan%,0.00%,0,0,0


In [46]:
# with pd.ExcelWriter('E:/myfile/订单日报数据/日报相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     df_dy.to_excel(writer, sheet_name='抖音渠道转化', index=False)  # index=False 表示不导出行索引  

## 总体 不含抖音渠道*****

In [47]:
#总体 不含抖音渠道
df1 = group_data(df[df["归属渠道"]!="抖音渠道"]) 
df1[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","风控通过件","风控通过率","出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]


,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,风控通过件,风控通过率,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
下单日期,,,,,,,,,,,,,,,
2024-11-04,2567,613,23.88%,581,22.63%,416,71.60%,165,149,25.65%,23,3.96%,87,14.97%,3.39%
2024-11-05,2516,583,23.17%,566,22.50%,405,71.55%,161,149,26.33%,19,3.36%,84,14.84%,3.34%
2024-11-06,2634,663,25.17%,535,20.31%,361,67.48%,174,138,25.79%,13,2.43%,80,14.95%,3.04%
2024-11-07,2662,640,24.04%,559,21.00%,362,64.76%,197,155,27.73%,17,3.04%,86,15.38%,3.23%
2024-11-08,2561,595,23.23%,536,20.93%,307,57.28%,229,160,29.85%,14,2.61%,80,14.93%,3.12%
2024-11-09,2576,677,26.28%,489,18.98%,330,67.48%,159,135,27.61%,11,2.25%,67,13.70%,2.60%
2024-11-10,2490,641,25.74%,500,20.08%,305,61.00%,195,172,34.40%,17,3.40%,80,16.00%,3.21%
2024-11-11,835,248,29.70%,127,15.21%,71,55.91%,56,39,30.71%,5,3.94%,5,3.94%,0.60%


# 免审订单转化统计

In [48]:
# 免审

df_ms = group_data(df[df["decision_result"].str.contains(pat="免人审",regex=False)==True]) 
df_ms[["是否进件","人审拒绝","客户取消","是否出库","待审核","进件出库率","取消率","人审拒绝率",'出库前风控强拒','出库前强拒比例','无法联系','无法联系占比']] #
#"无法联系占比",,"出库","进件出库率","取消率","人审拒绝率","订单出库率"

,是否进件,人审拒绝,客户取消,是否出库,待审核,进件出库率,取消率,人审拒绝率,出库前风控强拒,出库前强拒比例,无法联系,无法联系占比
下单日期,,,,,,,,,,,,
2024-11-04,144,15,33,80,0,55.56%,22.92%,10.42%,13,9.03%,2,1.39%
2024-11-05,133,9,37,69,0,51.88%,27.82%,6.77%,15,11.28%,2,1.50%
2024-11-06,138,23,31,71,0,51.45%,22.46%,16.67%,11,7.97%,0,0.00%
2024-11-07,140,18,35,69,0,49.29%,25.00%,12.86%,11,7.86%,2,1.43%
2024-11-08,130,15,22,68,4,52.31%,16.92%,11.54%,14,10.77%,0,0.00%
2024-11-09,121,9,24,56,14,46.28%,19.83%,7.44%,9,7.44%,1,0.83%
2024-11-10,150,3,27,75,18,50.00%,18.00%,2.00%,15,10.00%,0,0.00%
2024-11-11,36,6,3,4,18,11.11%,8.33%,16.67%,4,11.11%,0,0.00%


In [49]:
# with pd.ExcelWriter('E:/myfile/订单日报数据/日报相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     df_ms.to_excel(writer, sheet_name='免审整体转化', index=False)  # index=False 表示不导出行索引  

# 免押订单转化率

## 全免押

In [50]:
#全免押
df_my = group_data(df[df["押金类型"]=="全免押"]) 

df_my[["去重订单数","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝",
                    "风控通过件","风控通过率","客户取消","待审核","是否出库","进件出库率","取消率","人审拒绝率","订单出库率"]]#"前置拦截","拦截率",


,去重订单数,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,是否出库,进件出库率,取消率,人审拒绝率,订单出库率
下单日期,,,,,,,,,,,,,,,,
2024-11-04,2521,518,20.55%,354,68.34%,164,16,148,28.57%,34,0,88,16.99%,6.56%,3.09%,3.49%
2024-11-05,2496,532,21.31%,366,68.80%,166,15,151,28.38%,44,0,84,15.79%,8.27%,2.82%,3.37%
2024-11-06,2590,484,18.69%,312,64.46%,172,36,136,28.10%,37,0,80,16.53%,7.64%,7.44%,3.09%
2024-11-07,2608,503,19.29%,310,61.63%,193,41,152,30.22%,39,0,85,16.90%,7.75%,8.15%,3.26%
2024-11-08,2516,483,19.20%,264,54.66%,219,60,159,32.92%,36,6,79,16.36%,7.45%,12.42%,3.14%
2024-11-09,2517,430,17.08%,279,64.88%,151,19,132,30.70%,27,16,65,15.12%,6.28%,4.42%,2.58%
2024-11-10,2443,452,18.50%,262,57.96%,190,20,170,37.61%,32,22,80,17.70%,7.08%,4.42%,3.27%
2024-11-11,827,118,14.27%,62,52.54%,56,17,39,33.05%,4,24,5,4.24%,3.39%,14.41%,0.60%


In [51]:
# with pd.ExcelWriter('E:/myfile/订单日报数据/日报相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     df_qmy.to_excel(writer, sheet_name='全免押转化', index=False)  # index=False 表示不导出行索引  

## 部分免押

In [52]:
#部分免押
df_my = group_data(df[df["押金类型"]=="部分免押"]) 

df_my[["是否进件","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","取消率","人审拒绝率"]]#"出库","进件出库率",,"订单出库率""待审核","待审核","去重订单数","预授权通过率"

,是否进件,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,取消率,人审拒绝率
下单日期,,,,,,,,,,
2024-11-04,66,64,96.97%,2,0,2,3.03%,1,1.52%,0.00%
2024-11-05,45,44,97.78%,1,0,1,2.22%,0,0.00%,0.00%
2024-11-06,54,52,96.30%,2,0,2,3.70%,1,1.85%,0.00%
2024-11-07,57,52,91.23%,5,2,3,5.26%,0,0.00%,3.51%
2024-11-08,58,47,81.03%,11,9,2,3.45%,0,0.00%,15.52%
2024-11-09,60,52,86.67%,8,5,3,5.00%,1,1.67%,8.33%
2024-11-10,48,43,89.58%,5,3,2,4.17%,1,2.08%,6.25%
2024-11-11,9,9,100.00%,0,0,0,0.00%,0,0.00%,0.00%


In [53]:
# with pd.ExcelWriter('E:/myfile/订单日报数据/日报相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     df_bfmy.to_excel(writer, sheet_name='部分免押转化', index=False)  # index=False 表示不导出行索引  

# 风控拦截

In [54]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'auto_pay_tag', '下单日期', '月份',
       '拒绝理由', '取消原因', '电审拒绝原因', '无法联系原因', '商品类型', '租赁方案', '押金类型', '优惠券使用否',
       '是否进行预授权', '颜色', '内存', '进件', '来源渠道', '型号1', '型号_new', '归属渠道', '状态编码',
       '是否前置拦截', '是否机审强拒', '是否出库前风控强拒', '审核状态', '下单时段', '待审核', '前置拦截', '机审强拒',
       '人审拒绝', '客户取消', '出库前风控强拒', '无法联系', '是否进件', '是否出库', '进件前取消',
       'create_hour'],
      dtype='object')

## 前置拦截原因

In [55]:
def pre_reject(df):
    #年龄超过49岁或低于18岁
    df_nl=df[(df["拒绝理由"].notnull())&(df["status2"]=="订单取消")&(df["拒绝理由"].str.contains("年龄超过49岁或低于18岁"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_nl=df_nl.rename(columns={"order_id":"年龄超过49岁或低于18岁"})
    #法院被执行人
    df_zx=df[(df["拒绝理由"].notnull())&(df["status2"]=="订单取消")&(df["拒绝理由"].str.contains("被执行人"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_zx=df_zx.rename(columns={"order_id":"命中法院失信/限高被执行人"})

    df_nl_zx=pd.merge(df_nl,df_zx,on="下单日期",how='outer')

    df_xs=df[(df["拒绝理由"].notnull())&(df["status2"]=="订单取消")&(df["拒绝理由"].str.contains("命中刑事案件"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_xs=df_xs.rename(columns={"order_id":"命中刑事案件"})

    df_zx_xs=pd.merge(df_nl_zx,df_xs,on="下单日期",how='outer')

    #命中借贷纠纷
    df_jf=df[(df["拒绝理由"].notnull())&(df["status2"]=="订单取消")&(df["拒绝理由"].str.contains("命中借贷纠纷"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_jf=df_jf.rename(columns={"order_id":"命中借贷纠纷"})

    df_zx_xs_jf=pd.merge(df_zx_xs,df_jf,on="下单日期",how='outer')

    #命中融安分低于680强拒
    df_ra=df[(df["拒绝理由"].notnull())&(df["status2"]=="订单取消")&(df["拒绝理由"].str.contains("命中融安分低于680"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_ra=df_ra.rename(columns={"order_id":"命中融安分低于680"})

    df_zx_xs_jf_ra=pd.merge(df_zx_xs_jf,df_ra,on="下单日期",how='outer')

    #命中特殊名单验证-高风险
    df_gf=df[(df["拒绝理由"].notnull())&(df["status2"]=="订单取消")&(df["拒绝理由"].str.contains("命中特殊名单验证-高风险"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_gf=df_gf.rename(columns={"order_id":"命中高风险名单"})

    df_zx_xs_jf_ra_gf=pd.merge(df_zx_xs_jf_ra,df_gf,on="下单日期",how='outer')

    #身份证号码命中黑名单
    df_hm=df[(df["拒绝理由"].notnull())&(df["status2"]=="订单取消")&(df["拒绝理由"].str.contains("身份证"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_hm=df_hm.rename(columns={"order_id":"身份证命中黑名单"})
                                                                         
    df_zx_xs_jf_ra_gf_df_hm=pd.merge(df_zx_xs_jf_ra_gf,df_hm,on="下单日期",how='outer')

    #智融分低于458
    df_zr=df[(df["拒绝理由"].notnull())&(df["status2"]=="订单取消")&(df["拒绝理由"].str.contains("冰鉴火眸分"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_zr=df_zr.rename(columns={"order_id":"冰鉴火眸分<500"})

    df_zx_xs_jf_ra_gf_df_hm_zr=pd.merge(df_zx_xs_jf_ra_gf_df_hm,df_zr,on="下单日期",how='outer')

    #命中强拒加入临时黑名单
    df_lshm=df[(df["拒绝理由"].notnull())&(df["status2"]=="订单取消")&(df["拒绝理由"].str.contains("命中强拒加入临时黑名单"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_lshm=df_lshm.rename(columns={"order_id":"命中强拒加入临时黑名单"})

    df_zx_xs_jf_ra_gf_df_hm_zr_lshm=pd.merge(df_zx_xs_jf_ra_gf_df_hm_zr,df_lshm,on="下单日期",how='outer')

    #当日下单次数大于等于5次
    df_cf=df[(df["拒绝理由"].notnull())&(df["status2"]=="订单取消")&(df["拒绝理由"].str.contains("当日下单次数大于等于5次"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_cf=df_cf.rename(columns={"order_id":"当日下单次数大于等于5次"})

    df_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf=pd.merge(df_zx_xs_jf_ra_gf_df_hm_zr_lshm,df_cf,on="下单日期",how='outer')

    #当月下单次数大于等于10次
    df_cf2=df[(df["total_describes"].notnull())&(df["status2"]=="订单取消")&(df["total_describes"].str.contains("当月下单次数大于等于10次"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_cf2=df_cf2.rename(columns={"order_id":"当月下单次数大于等于10次"})

    f_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf_cf2=pd.merge(df_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf,df_cf2,on="下单日期",how='outer')

    #命中特殊地区 新建|西藏
    df_diqu=df[(df["total_describes"].notnull())&(df["status2"]=="订单取消")&(df["total_describes"].str.contains("命中特殊地区"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_diqu=df_diqu.rename(columns={"order_id":"命中特殊地区 新疆|西藏"})
    f_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf_cf2_diqu=pd.merge(f_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf_cf2,df_diqu,on="下单日期",how='outer')

    return f_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf_cf2_diqu

In [56]:
df_pre_reject_all = pre_reject(df)
list_reg=["下单日期","年龄超过49岁或低于18岁","身份证命中黑名单","命中法院失信/限高被执行人","命中刑事案件","命中借贷纠纷","命中高风险名单","命中融安分低于680","冰鉴火眸分<500"
    ,"命中强拒加入临时黑名单","当日下单次数大于等于5次","当月下单次数大于等于10次","命中特殊地区 新疆|西藏"]

df_pre_reject = df_pre_reject_all[list_reg].set_index("下单日期")  #.sort_values(by=["下单日期"])    #[10:]
df_pre_reject

,年龄超过49岁或低于18岁,身份证命中黑名单,命中法院失信/限高被执行人,命中刑事案件,命中借贷纠纷,命中高风险名单,命中融安分低于680,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏
下单日期,,,,,,,,,,,,
2024-11-04,48,NaN,93,1.0,2.0,8,NaN,354,81,6,2,31
2024-11-05,56,NaN,82,2.0,2.0,10,NaN,335,71,5,3,27
2024-11-06,55,NaN,105,4.0,4.0,6,NaN,367,91,4,6,39
2024-11-07,61,NaN,103,1.0,NaN,11,NaN,354,85,7,6,40
2024-11-08,58,NaN,96,1.0,3.0,9,NaN,346,70,1,4,32
2024-11-09,73,NaN,137,3.0,1.0,13,NaN,355,77,5,8,42
2024-11-10,73,NaN,111,3.0,1.0,7,NaN,358,71,9,7,39
2024-11-11,23,NaN,51,NaN,1.0,2,NaN,131,29,4,4,10


In [57]:
# with pd.ExcelWriter('E:/myfile/订单日报数据/日报相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     df_pre_reject.to_excel(writer, sheet_name='前置拦截原因分布', index=False)  # index=False 表示不导出行索引  

### 搜索渠道

In [58]:
df_dy = df[df.归属渠道=='搜索渠道']
df_pre_reject_dy = pre_reject(df_dy)
list_reg=["下单日期","年龄超过49岁或低于18岁","身份证命中黑名单","命中法院失信/限高被执行人","命中刑事案件","命中借贷纠纷","命中高风险名单","命中融安分低于680","冰鉴火眸分<500"
    ,"命中强拒加入临时黑名单","当日下单次数大于等于5次","当月下单次数大于等于10次","命中特殊地区 新疆|西藏"]

df_pre_reject = df_pre_reject_dy[list_reg].set_index('下单日期') #.sort_values(by=["下单日期"])#[10:]
df_pre_reject

,年龄超过49岁或低于18岁,身份证命中黑名单,命中法院失信/限高被执行人,命中刑事案件,命中借贷纠纷,命中高风险名单,命中融安分低于680,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏
下单日期,,,,,,,,,,,,
2024-11-04,6,NaN,24,NaN,NaN,1.0,NaN,1.0,19,1.0,1.0,5
2024-11-05,11,NaN,20,1.0,1.0,3.0,NaN,1.0,20,NaN,NaN,7
2024-11-06,9,NaN,26,1.0,1.0,2.0,NaN,NaN,30,1.0,2.0,6
2024-11-07,11,NaN,24,NaN,NaN,1.0,NaN,2.0,14,1.0,3.0,8
2024-11-08,9,NaN,19,1.0,3.0,NaN,NaN,NaN,27,NaN,NaN,4
2024-11-09,10,NaN,25,1.0,NaN,2.0,NaN,NaN,29,1.0,2.0,3
2024-11-10,8,NaN,20,1.0,NaN,2.0,NaN,1.0,23,1.0,1.0,2
2024-11-11,1,NaN,7,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,1


### 芝麻渠道

In [59]:
df_dy = df[df.归属渠道=='芝麻租物']
df_pre_reject_dy = pre_reject(df_dy)
list_reg=["下单日期","年龄超过49岁或低于18岁","身份证命中黑名单","命中法院失信/限高被执行人","命中刑事案件","命中借贷纠纷","命中高风险名单","命中融安分低于680","冰鉴火眸分<500"
    ,"命中强拒加入临时黑名单","当日下单次数大于等于5次","当月下单次数大于等于10次","命中特殊地区 新疆|西藏"]

df_pre_reject = df_pre_reject_dy[list_reg].set_index('下单日期') #.sort_values(by=["下单日期"])#[10:]
df_pre_reject

,年龄超过49岁或低于18岁,身份证命中黑名单,命中法院失信/限高被执行人,命中刑事案件,命中借贷纠纷,命中高风险名单,命中融安分低于680,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏
下单日期,,,,,,,,,,,,
2024-11-04,32,NaN,55,NaN,1.0,6,NaN,232,29,4.0,NaN,22
2024-11-05,32,NaN,48,NaN,1.0,6,NaN,192,37,3.0,3.0,10
2024-11-06,39,NaN,54,3.0,1.0,3,NaN,215,35,NaN,2.0,19
2024-11-07,36,NaN,59,1.0,NaN,6,NaN,216,49,4.0,1.0,20
2024-11-08,38,NaN,57,NaN,NaN,7,NaN,199,26,1.0,3.0,16
2024-11-09,46,NaN,82,2.0,1.0,8,NaN,234,34,3.0,4.0,31
2024-11-10,50,NaN,73,1.0,1.0,3,NaN,264,28,6.0,4.0,24
2024-11-11,19,NaN,38,NaN,1.0,1,NaN,101,17,4.0,3.0,7


### 抖音渠道*****zj放开

In [60]:
# df_dy = df[df.归属渠道=='抖音渠道']
# df_pre_reject_dy = pre_reject(df_dy)
# list_reg=["下单日期","年龄超过49岁或低于18岁","身份证命中黑名单","命中法院失信/限高被执行人","命中刑事案件","命中借贷纠纷","命中高风险名单","命中融安分低于680","冰鉴火眸分<500"
#     ,"命中强拒加入临时黑名单","当日下单次数大于等于5次","当月下单次数大于等于10次","命中特殊地区 新疆|西藏"]

# df_pre_reject = df_pre_reject_dy[list_reg].set_index('下单日期') #.sort_values(by=["下单日期"])#[10:]
# df_pre_reject

## 机审强拒原因

In [61]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'auto_pay_tag', '下单日期', '月份',
       '拒绝理由', '取消原因', '电审拒绝原因', '无法联系原因', '商品类型', '租赁方案', '押金类型', '优惠券使用否',
       '是否进行预授权', '颜色', '内存', '进件', '来源渠道', '型号1', '型号_new', '归属渠道', '状态编码',
       '是否前置拦截', '是否机审强拒', '是否出库前风控强拒', '审核状态', '下单时段', '待审核', '前置拦截', '机审强拒',
       '人审拒绝', '客户取消', '出库前风控强拒', '无法联系', '是否进件', '是否出库', '进件前取消',
       'create_hour'],
      dtype='object')

In [62]:
def model_reject(df_j):
    #评分等级D且综合风险等级为3
    df_j1=df_j[df_j["拒绝理由"].str.contains(pat="评分等级D且综合风险等级为3",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j1=df_j1.rename(columns={"order_id":"评分等级D且综合风险等级为3"})

    #综合风险等级为3且非免押
    df_j2=df_j[df_j["拒绝理由"].str.contains(pat="综合风险等级为3",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j2=df_j2.rename(columns={"order_id":"综合风险等级为3"})

    df_j12=pd.merge(df_j1,df_j2,on="下单日期",how='outer')

    #法院被执行人
    df_j3=df_j[df_j["拒绝理由"].str.contains(pat="被执行人",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j3=df_j3.rename(columns={"order_id":"命中法院失信/限高被执行人"})

    df_j123=pd.merge(df_j12,df_j3,on="下单日期",how='outer')

    df_j4=df_j[df_j["拒绝理由"].str.contains(pat="命中刑事案件",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j4=df_j4.rename(columns={"order_id":"命中刑事案件"})

    df_j1234=pd.merge(df_j123,df_j4,on="下单日期",how='outer')

    #命中借贷纠纷
    df_j5=df_j[df_j["拒绝理由"].str.contains(pat="命中借贷纠纷",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j5=df_j5.rename(columns={"order_id":"命中借贷纠纷"})

    df_j12345=pd.merge(df_j1234,df_j5,on="下单日期",how='outer')

    #命中融安分低于680强拒
    df_j6=df_j[df_j["拒绝理由"].str.contains(pat="命中融安分低于680",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j6=df_j6.rename(columns={"order_id":"命中融安分低于680"})

    df_j123456=pd.merge(df_j12345,df_j6,on="下单日期",how='outer')

    #命中特殊名单验证-高风险
    df_j7=df_j[df_j["拒绝理由"].str.contains(pat="命中特殊名单验证-高风险",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j7=df_j7.rename(columns={"order_id":"命中高风险名单"})

    df_j1234567=pd.merge(df_j123456,df_j7,on="下单日期",how='outer')

    #命中风险勘测名单
    df_j8=df_j[df_j["拒绝理由"].str.contains(pat="风险勘测",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j8=df_j8.rename(columns={"order_id":"命中风险勘测"})

    df_j12345678=pd.merge(df_j1234567,df_j8,on="下单日期",how='outer')

    #蚁盾分>=90
    df_j9=df_j[df_j["拒绝理由"].str.contains(pat="蚁盾分>=80",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j9=df_j9.rename(columns={"order_id":"蚁盾分>=80"})

    df_j123456789=pd.merge(df_j12345678,df_j9,on="下单日期",how='outer')

    #30天多头>=10且90天多头>=31
    df_j10=df_j[df_j["拒绝理由"].str.contains(pat="命中30天多头>13并且90天多头>39",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j10=df_j10.rename(columns={"order_id":"命中30天多头>13并且90天多头>39"})

    df_j12345678910=pd.merge(df_j123456789,df_j10,on="下单日期",how='outer')

    #履约历史等级1且为搜索渠道
    df_j11=df_j[df_j["拒绝理由"].str.contains(pat="履约历史等级1",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j11=df_j11.rename(columns={"order_id":"履约历史等级1且为搜索渠道"})

    df_j1234567891011=pd.merge(df_j12345678910,df_j11,on="下单日期",how='outer')

    #云商分低于515
    df_j12=df_j[df_j["拒绝理由"].str.contains(pat="云商分低于515",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j12=df_j12.rename(columns={"order_id":"云商分低于515"})

    df_j123456789101112=pd.merge(df_j1234567891011,df_j12,on="下单日期",how='outer')

    #评分等级低于D
    df_j12=df_j[df_j["拒绝理由"].str.contains(pat="评分等级低于D",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j12=df_j12.rename(columns={"order_id":"评分等级低于D"})

    df_dj=pd.merge(df_j123456789101112,df_j12,on="下单日期",how='outer')

    #评分等级D且非免押客户
    df_j13=df_j[df_j["拒绝理由"].str.contains(pat="评分等级D且非免押客户",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j13=df_j13.rename(columns={"order_id":"评分等级D且非免押客户"})

    df_dj_fmy=pd.merge(df_dj,df_j13,on="下单日期",how='outer')

    #非免押客户
    df_j14=df_j[df_j["拒绝理由"].str.contains(pat="命中非免押用户拒绝",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j14=df_j14.rename(columns={"order_id":"非免押用户"})

    df_dj_last1=pd.merge(df_dj_fmy,df_j14,on="下单日期",how='outer')

    #命中自有模型强拒
    df_j15=df_j[df_j["拒绝理由"].str.contains(pat="命中自有模型强拒",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j15=df_j15.rename(columns={"order_id":"命中自有模型强拒"})

    df_dj_last2=pd.merge(df_dj_last1,df_j15,on="下单日期",how='outer')

    #非芝麻租物进件且履约历史等级=1强拒
    df_j16=df_j[df_j["拒绝理由"].str.contains(pat="履约历史等级=1强拒",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j16=df_j16.rename(columns={"order_id":"非芝麻租物进件且履约历史等级=1强拒"})

    df_dj_last3=pd.merge(df_dj_last2,df_j16,on="下单日期",how='outer')

    # 决策树组合策略 14
    df_j17=df_j[df_j["拒绝理由"].str.contains(pat="命中策略240703_14强拒",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j17=df_j17.rename(columns={"order_id":"命中策略240703_14强拒"})

    df_dj_last4=pd.merge(df_dj_last3,df_j17,on="下单日期",how='outer')

    # 决策树组合策略 4
    df_j18=df_j[df_j["拒绝理由"].str.contains(pat="命中策略240703_4强拒",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j18=df_j18.rename(columns={"order_id":"命中策略240703_4强拒"})

    df_dj_last5=pd.merge(df_dj_last4,df_j18,on="下单日期",how='outer')

    # 决策树组合策略 10 
    df_j19=df_j[df_j["拒绝理由"].str.contains(pat="命中策略240703_10强拒",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j19=df_j19.rename(columns={"order_id":"命中策略240703_10强拒"})

    df_dj_last6 = pd.merge(df_dj_last5,df_j19,on="下单日期",how='outer')


    #蚂蚁数控风险等级=996强拒
    df_j21 =df_j[df_j["total_describes"].str.contains(pat="蚂蚁数控风险等级=996强拒",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j21=df_j21.rename(columns={"order_id":"蚂蚁数控风险等级=996强拒"})

    df_dj_last8 =pd.merge(df_dj_last6,df_j21,on="下单日期",how='outer')

    #蚂蚁数控风险等级=998强拒
    df_j22 =df_j[df_j["total_describes"].str.contains(pat="蚂蚁数控风险等级=998强拒",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j22 =df_j22.rename(columns={"order_id":"蚂蚁数控风险等级=998强拒"})

    df_dj_last9 =pd.merge(df_dj_last8,df_j22,on="下单日期",how='outer')

    # 百融裁判文书命中拒绝或者fico分拒绝
    df_j23 =df_j[df_j["result"].str.contains(pat="命中出库前风控流强拒",regex=False)==True].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_j23 =df_j23.rename(columns={"order_id":"命中出库前风控流强拒"})

    df_dj_last10 =pd.merge(df_dj_last9,df_j23,on="下单日期",how='outer')


    return df_dj_last10

## 总体机审强拒原因

In [63]:
df_j = df[df["进件"]=="进件"]
df_model_reject_all = model_reject(df_j)

df_model_reject_all.sort_values(by=["下单日期"])

df_model_reject_all.set_index(keys=['下单日期'],inplace=True)

df_model_reject_all

,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人,命中刑事案件,命中借贷纠纷,命中融安分低于680,命中高风险名单,命中风险勘测,蚁盾分>=80,命中30天多头>13并且90天多头>39,...,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒
下单日期,,,,,,,,,,,,,,,,,,,,,
2024-11-04,3.0,5.0,5,2.0,2.0,34,NaN,NaN,28,NaN,...,NaN,63,221,46,31,30,NaN,7,1.0,15
2024-11-05,2.0,4.0,8,NaN,3.0,37,NaN,NaN,31,NaN,...,NaN,44,225,33,38,41,NaN,1,NaN,20
2024-11-06,NaN,4.0,6,NaN,4.0,32,NaN,NaN,30,NaN,...,NaN,52,195,23,21,18,NaN,1,NaN,12
2024-11-07,1.0,5.0,7,NaN,4.0,41,NaN,NaN,26,NaN,...,NaN,52,194,30,21,29,NaN,2,NaN,15
2024-11-08,1.0,2.0,7,NaN,NaN,35,NaN,NaN,22,NaN,...,NaN,45,166,22,14,20,NaN,2,1.0,11
2024-11-09,1.0,4.0,2,1.0,1.0,35,NaN,NaN,29,NaN,...,NaN,52,175,25,19,21,NaN,2,1.0,8
2024-11-10,1.0,2.0,4,1.0,2.0,36,NaN,NaN,33,NaN,...,NaN,43,161,30,17,17,NaN,1,1.0,15
2024-11-11,NaN,NaN,3,1.0,NaN,6,NaN,NaN,7,NaN,...,NaN,9,36,4,1,5,NaN,1,1.0,3


In [64]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx', engine='xlsxwriter') as writer:    #, mode='a'
    # 将df写入新的sheet，名为'new_sheet'  
    df_model_reject_all.to_excel(writer, sheet_name='总体机审强制拒绝原因', index=True)  # index=False 表示不导出行索引  

## 芝麻租物机审强拒原因

In [65]:
# df_j = df[df["进件"]=="进件"] 
df_zm=df[(df["归属渠道"]=="芝麻租物")&(df["进件"]=="进件")]
df_model_reject_zm = model_reject(df_zm)

df_model_reject_zm.sort_values(by=["下单日期"])

df_model_reject_zm.set_index(keys=['下单日期'],inplace=True)

df_model_reject_zm

,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人,命中刑事案件,命中借贷纠纷,命中融安分低于680,命中高风险名单,命中风险勘测,蚁盾分>=80,命中30天多头>13并且90天多头>39,...,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒
下单日期,,,,,,,,,,,,,,,,,,,,,
2024-11-04,2.0,3.0,2.0,2.0,1.0,8,NaN,NaN,13,NaN,...,NaN,25,62,6.0,11,7,NaN,1.0,NaN,10
2024-11-05,1.0,2.0,5.0,NaN,NaN,14,NaN,NaN,14,NaN,...,NaN,21,83,3.0,15,20,NaN,NaN,NaN,6
2024-11-06,NaN,3.0,2.0,NaN,3.0,13,NaN,NaN,11,NaN,...,NaN,23,75,NaN,14,9,NaN,NaN,NaN,8
2024-11-07,1.0,3.0,4.0,NaN,1.0,14,NaN,NaN,11,NaN,...,NaN,26,76,NaN,11,14,NaN,NaN,NaN,5
2024-11-08,NaN,NaN,1.0,NaN,NaN,15,NaN,NaN,12,NaN,...,NaN,24,50,2.0,4,7,NaN,NaN,NaN,7
2024-11-09,NaN,1.0,NaN,NaN,NaN,9,NaN,NaN,11,NaN,...,NaN,28,59,1.0,9,7,NaN,NaN,NaN,3
2024-11-10,NaN,NaN,2.0,NaN,2.0,12,NaN,NaN,19,NaN,...,NaN,23,57,2.0,10,7,NaN,NaN,NaN,9
2024-11-11,NaN,NaN,3.0,1.0,NaN,6,NaN,NaN,5,NaN,...,NaN,4,17,2.0,1,3,NaN,NaN,NaN,3


In [66]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
    # 将df写入新的sheet，名为'new_sheet'  
    df_model_reject_zm.to_excel(writer, sheet_name='芝麻渠道机审强制拒绝原因', index=True)  # index=False 表示不导出行索引  

## 搜索渠道机审强拒原因

In [67]:
# df_j = df[df["进件"]=="进件"] 
df_ss=df[(df["归属渠道"]=="搜索渠道")&(df["进件"]=="进件")]
df_model_reject_ss = model_reject(df_ss)

df_model_reject_ss.sort_values(by=["下单日期"])

df_model_reject_ss1 = df_model_reject_ss.drop(columns = ['命中刑事案件'])
df_model_reject_ss1.set_index(keys=['下单日期'],inplace=True)
df_model_reject_ss1

,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人,命中借贷纠纷,命中融安分低于680,命中高风险名单,命中风险勘测,蚁盾分>=80,命中30天多头>13并且90天多头>39,履约历史等级1且为搜索渠道,...,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒
下单日期,,,,,,,,,,,,,,,,,,,,,
2024-11-04,1.0,2.0,2.0,1.0,15.0,NaN,NaN,10,NaN,23,...,NaN,23,114,23,14.0,18,NaN,NaN,1.0,2.0
2024-11-05,1.0,1.0,3.0,1.0,18.0,NaN,NaN,8,NaN,22,...,NaN,14,114,22,18.0,17,NaN,NaN,NaN,6.0
2024-11-06,NaN,1.0,3.0,NaN,10.0,NaN,NaN,4,NaN,11,...,NaN,18,73,11,3.0,6,NaN,NaN,NaN,2.0
2024-11-07,NaN,2.0,2.0,1.0,18.0,NaN,NaN,6,NaN,22,...,NaN,15,87,22,7.0,14,NaN,NaN,NaN,1.0
2024-11-08,1.0,1.0,3.0,NaN,13.0,NaN,NaN,2,NaN,9,...,NaN,8,86,9,8.0,13,NaN,NaN,1.0,2.0
2024-11-09,1.0,1.0,NaN,1.0,15.0,NaN,NaN,9,NaN,12,...,NaN,11,71,12,3.0,9,NaN,NaN,1.0,3.0
2024-11-10,1.0,1.0,2.0,NaN,16.0,NaN,NaN,6,NaN,14,...,NaN,14,83,14,4.0,7,NaN,NaN,1.0,2.0
2024-11-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,2,...,NaN,3,11,2,NaN,2,NaN,NaN,NaN,NaN


In [68]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
    # 将df写入新的sheet，名为'new_sheet'  
    df_model_reject_ss1.to_excel(writer, sheet_name='搜索渠道机审强制拒绝原因', index=True)  # index=False 表示不导出行索引  

## 抖音渠道机审强拒原因

### 抖音渠道 前置拦截 后置到 机审

In [69]:
def pre_reject_dy(df):
    #年龄超过49岁或低于18岁
    df_nl=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("年龄超过49岁或低于18岁"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_nl=df_nl.rename(columns={"order_id":"年龄超过49岁或低于18岁"})
    #法院被执行人
    df_zx=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("被执行人"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_zx=df_zx.rename(columns={"order_id":"命中法院失信/限高被执行人"})

    df_nl_zx=pd.merge(df_nl,df_zx,on="下单日期",how='outer')

    df_xs=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("命中刑事案件"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_xs=df_xs.rename(columns={"order_id":"命中刑事案件"})

    df_zx_xs=pd.merge(df_nl_zx,df_xs,on="下单日期",how='outer')

    #命中借贷纠纷
    df_jf=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("命中借贷纠纷"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_jf=df_jf.rename(columns={"order_id":"命中借贷纠纷"})

    df_zx_xs_jf=pd.merge(df_zx_xs,df_jf,on="下单日期",how='outer')

    #命中融安分低于680强拒
    df_ra=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("命中融安分低于680"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_ra=df_ra.rename(columns={"order_id":"命中融安分低于680"})

    df_zx_xs_jf_ra=pd.merge(df_zx_xs_jf,df_ra,on="下单日期",how='outer')

    #命中特殊名单验证-高风险
    df_gf=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("命中特殊名单验证-高风险"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_gf=df_gf.rename(columns={"order_id":"命中高风险名单"})

    df_zx_xs_jf_ra_gf=pd.merge(df_zx_xs_jf_ra,df_gf,on="下单日期",how='outer')

    #身份证号码命中黑名单
    df_hm=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("身份证"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_hm=df_hm.rename(columns={"order_id":"身份证命中黑名单"})
                                                                         
    df_zx_xs_jf_ra_gf_df_hm=pd.merge(df_zx_xs_jf_ra_gf,df_hm,on="下单日期",how='outer')

    #智融分低于458
    df_zr=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("冰鉴火眸分"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_zr=df_zr.rename(columns={"order_id":"冰鉴火眸分<500"})

    df_zx_xs_jf_ra_gf_df_hm_zr=pd.merge(df_zx_xs_jf_ra_gf_df_hm,df_zr,on="下单日期",how='outer')

    #命中强拒加入临时黑名单
    df_lshm=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("命中强拒加入临时黑名单"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_lshm=df_lshm.rename(columns={"order_id":"命中强拒加入临时黑名单"})

    df_zx_xs_jf_ra_gf_df_hm_zr_lshm=pd.merge(df_zx_xs_jf_ra_gf_df_hm_zr,df_lshm,on="下单日期",how='outer')

    #当日下单次数大于等于5次
    df_cf=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("当日下单次数大于等于5次"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_cf=df_cf.rename(columns={"order_id":"当日下单次数大于等于5次"})

    df_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf=pd.merge(df_zx_xs_jf_ra_gf_df_hm_zr_lshm,df_cf,on="下单日期",how='outer')

    #当月下单次数大于等于10次
    df_cf2=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("当月下单次数大于等于10次"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_cf2=df_cf2.rename(columns={"order_id":"当月下单次数大于等于10次"})

    f_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf_cf2=pd.merge(df_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf,df_cf2,on="下单日期",how='outer')

    #命中特殊地区 新建|西藏
    df_diqu=df[(df["total_describes"].notnull())&(df["total_describes"].str.contains("命中特殊地区"))].groupby(["下单日期"]).agg({"order_id":np.size}).reset_index()
    df_diqu=df_diqu.rename(columns={"order_id":"命中特殊地区 新疆|西藏"})
    f_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf_cf2_diqu=pd.merge(f_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf_cf2,df_diqu,on="下单日期",how='outer')

    return f_zx_xs_jf_ra_gf_df_hm_zr_lshm_cf_cf2_diqu

In [70]:
df_dy = df[df.归属渠道=='抖音渠道']
df_pre_reject_dy = pre_reject_dy(df_dy)
list_reg=["下单日期","年龄超过49岁或低于18岁","身份证命中黑名单","命中刑事案件","命中借贷纠纷","命中高风险名单","命中融安分低于680","冰鉴火眸分<500"
    ,"命中强拒加入临时黑名单","当日下单次数大于等于5次","当月下单次数大于等于10次","命中特殊地区 新疆|西藏"]   # ,"命中法院失信/限高被执行人"

df_pre_reject = df_pre_reject_dy[list_reg].set_index('下单日期') #.sort_values(by=["下单日期"])#[10:]
df_pre_reject

,年龄超过49岁或低于18岁,身份证命中黑名单,命中刑事案件,命中借贷纠纷,命中高风险名单,命中融安分低于680,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏
下单日期,,,,,,,,,,,
2024-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2024-11-06,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2024-11-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


### 机审强拒

In [71]:
# df_j = df[df["进件"]=="进件"] 
df_dy=df[(df["归属渠道"]=="抖音渠道")&(df["进件"]=="进件")]
df_model_reject_dy = model_reject(df_dy)

df_model_reject_dy.sort_values(by=["下单日期"])

df_model_reject_dy1 = df_model_reject_dy.drop(columns = ['命中刑事案件'])
df_model_reject_dy1.set_index(keys=['下单日期'],inplace=True)
df_model_reject_dy1

,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人,命中借贷纠纷,命中融安分低于680,命中高风险名单,命中风险勘测,蚁盾分>=80,命中30天多头>13并且90天多头>39,履约历史等级1且为搜索渠道,...,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒
下单日期,,,,,,,,,,,,,,,,,,,,,
2024-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2024-11-06,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
    # 将df写入新的sheet，名为'new_sheet'  
    df_model_reject_dy1.to_excel(writer, sheet_name='抖音渠道机审强制拒绝原因', index=True)  # index=False 表示不导出行索引  

In [73]:
# df_j[df_j["备注信息"].str.contains(pat="命中出库前风控流强拒",regex=False)==True].groupby(["下单日期","归属渠道"]).agg({"单号":np.size}).unstack()


In [74]:

# df_j[df_j["备注信息"].str.contains(pat="蚂蚁数控风险等级=996强拒",regex=False)==True].groupby(["下单日期","来源渠道"]).agg({"单号":np.size}).unstack()


In [75]:

# df_j[df_j["拒绝理由"].str.contains(pat="履约历史等级=1强拒",regex=False)==True].groupby(["下单日期","来源渠道"]).agg({"单号":np.size}).unstack()

# 不同型号机型情况

In [76]:
# 包含待审核
def crosstab_merge_xinghao(df0):
    df1=pd.crosstab(df0["型号_new"],df0["订单状态"],margins=True)
    df1=df1.rename(columns={"All":"去重订单数"})

    df2=pd.crosstab(df0["型号_new"],df0["进件"],margins=True)

    df1_2_merge=pd.merge(df1,df2["进件"],on="型号_new",how='outer')

    df3=pd.crosstab(df0["型号_new"],df0["审核状态"],margins=True)

    df123_merge=pd.merge(df1_2_merge,df3[["前置拦截","机审强拒","人审拒绝","客户取消","无法联系","出库前风控强拒","待审核"]],on="型号_new",how='outer')#"无法联系","待审核","待审核"
    df123_merge=df123_merge.rename(columns={"待审核_x":"合计待审核","待审核_y":"待审核"})

    df123_merge["拦截率"]=df123_merge["前置拦截"]/df123_merge["去重订单数"]
    df123_merge["拦截率"]=df123_merge["拦截率"].apply(lambda x:format(x,".2%"))

    df123_merge["预授权通过率"]=df123_merge["进件"]/df123_merge["去重订单数"]
    df123_merge["预授权通过率"]=df123_merge["预授权通过率"].apply(lambda x:format(x,".2%"))
    df123_merge["进件率"]=df123_merge["预授权通过率"]

    df123_merge["强拒比例"]=df123_merge["机审强拒"]/df123_merge["进件"]
    df123_merge["强拒比例"]=df123_merge["强拒比例"].apply(lambda x:format(x,".2%"))

    df123_merge["机审通过件"]=df123_merge["进件"]-df123_merge["机审强拒"]

    df123_merge["风控通过件"]=df123_merge["进件"]-df123_merge["机审强拒"]-df123_merge["人审拒绝"]
    df123_merge["风控通过率"]=df123_merge["风控通过件"]/df123_merge["进件"]
    df123_merge["风控通过率"]=df123_merge["风控通过率"].apply(lambda x:format(x,".2%"))

    df123_merge["取消率"]=df123_merge["客户取消"]/df123_merge["进件"]
    df123_merge["取消率"]=df123_merge["取消率"].apply(lambda x:format(x,".2%"))

    df123_merge["无法联系占比"]=df123_merge["无法联系"]/df123_merge["进件"]
    df123_merge["无法联系占比"]=df123_merge["无法联系占比"].apply(lambda x:format(x,".2%"))

    df123_merge["人审拒绝率"]=df123_merge["人审拒绝"]/df123_merge["进件"]
    df123_merge["人审拒绝率"]=df123_merge["人审拒绝率"].apply(lambda x:format(x,".2%"))

    df123_merge["出库前强拒比例"]=df123_merge["出库前风控强拒"]/df123_merge["进件"]
    df123_merge["出库前强拒比例"]=df123_merge["出库前强拒比例"].apply(lambda x:format(x,".2%"))

    df123_merge["出库"]=df123_merge["待发货"]+df123_merge["待收货"]+df123_merge["租赁中"]#df123_merge["待发货"]+df123_merge["待收货"]+
    df123_merge["进件出库率"]=df123_merge["出库"]/df123_merge["进件"]
    df123_merge["进件出库率"]=df123_merge["进件出库率"].apply(lambda x:format(x,".2%"))

    df123_merge["订单出库率"]=df123_merge["出库"]/df123_merge["去重订单数"]
    df123_merge["订单出库率"]=df123_merge["订单出库率"].apply(lambda x:format(x,".2%"))
    return df123_merge


In [77]:
# df0=df[df["归属渠道"]=="搜索渠道"]
# df_merge_xinghao = crosstab_merge_xinghao(df)
# df_xinghao = df_merge_xinghao[["去重订单数","前置拦截","拦截率","进件","进件率","机审强拒","强拒比例","机审通过件",
#                     "风控通过件","风控通过率","出库前风控强拒","出库前强拒比例"]]
# #"无法联系占比",,"出库","进件出库率","取消率","人审拒绝率","订单出库率"
# df_xinghao

In [78]:
# import pandas as pd
# test_data = pd.DataFrame()
# test_data.to_excel(f'E:/myfile/数据提取及分析/各机型逾期表现数据_{Today}.xlsx')

In [79]:
# test_data = pd.DataFram()
# test_data.to_excel('E:/myfile/数据提取及分析/各机型逾期表现数据_{Today}.xlsx')
# with pd.ExcelWriter('E:/myfile/数据提取及分析/各机型逾期表现数据_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     df_xinghao.to_excel(writer, sheet_name='近三月-各机型整体转化表现', index=True)  # index=False 表示不导出行索引  

In [80]:
# df_zw =df[df["归属渠道"]=="芝麻租物"]
# df_merge_xinghao_wu = crosstab_merge_xinghao(df_zw)
# df_xinghao_zw = df_merge_xinghao_wu[["去重订单数","前置拦截","拦截率","进件","进件率","机审强拒","强拒比例","机审通过件",
#                     "风控通过件","风控通过率","出库前风控强拒","出库前强拒比例"]]
# #"无法联系占比",,"出库","进件出库率","取消率","人审拒绝率","订单出库率"
# df_xinghao_zw

In [81]:
# with pd.ExcelWriter('E:/myfile/数据提取及分析/各机型逾期表现数据_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
#     df_xinghao_zw.to_excel(writer, sheet_name='芝麻渠道-近三月-各机型整体转化表现', index=True)  # index=False 表示不导出行索引  

In [82]:
# 不包含待审核
def crosstab_merge_xinghao1(df0):
    df1=pd.crosstab(df0["型号_new"],df0["订单状态"],margins=True)
    df1=df1.rename(columns={"All":"去重订单数"})

    df2=pd.crosstab(df0["型号_new"],df0["进件"],margins=True)

    df1_2_merge=pd.merge(df1,df2["进件"],on="型号_new",how='outer')

    df3=pd.crosstab(df0["型号_new"],df0["审核状态"],margins=True)

    df123_merge=pd.merge(df1_2_merge,df3[["前置拦截","机审强拒","人审拒绝","客户取消","无法联系","出库前风控强拒"]],on="型号_new",how='outer')#"无法联系","待审核","待审核"
    # df123_merge=df123_merge.rename(columns={"待审核_x":"合计待审核","待审核_y":"待审核"})

    df123_merge["拦截率"]=df123_merge["前置拦截"]/df123_merge["去重订单数"]
    df123_merge["拦截率"]=df123_merge["拦截率"].apply(lambda x:format(x,".2%"))

    df123_merge["预授权通过率"]=df123_merge["进件"]/df123_merge["去重订单数"]
    df123_merge["预授权通过率"]=df123_merge["预授权通过率"].apply(lambda x:format(x,".2%"))
    df123_merge["进件率"]=df123_merge["预授权通过率"]

    df123_merge["强拒比例"]=df123_merge["机审强拒"]/df123_merge["进件"]
    df123_merge["强拒比例"]=df123_merge["强拒比例"].apply(lambda x:format(x,".2%"))

    df123_merge["机审通过件"]=df123_merge["进件"]-df123_merge["机审强拒"]

    df123_merge["风控通过件"]=df123_merge["进件"]-df123_merge["机审强拒"]-df123_merge["人审拒绝"]
    df123_merge["风控通过率"]=df123_merge["风控通过件"]/df123_merge["进件"]
    df123_merge["风控通过率"]=df123_merge["风控通过率"].apply(lambda x:format(x,".2%"))

    df123_merge["取消率"]=df123_merge["客户取消"]/df123_merge["进件"]
    df123_merge["取消率"]=df123_merge["取消率"].apply(lambda x:format(x,".2%"))

    df123_merge["无法联系占比"]=df123_merge["无法联系"]/df123_merge["进件"]
    df123_merge["无法联系占比"]=df123_merge["无法联系占比"].apply(lambda x:format(x,".2%"))

    df123_merge["人审拒绝率"]=df123_merge["人审拒绝"]/df123_merge["进件"]
    df123_merge["人审拒绝率"]=df123_merge["人审拒绝率"].apply(lambda x:format(x,".2%"))

    df123_merge["出库前强拒比例"]=df123_merge["出库前风控强拒"]/df123_merge["进件"]
    df123_merge["出库前强拒比例"]=df123_merge["出库前强拒比例"].apply(lambda x:format(x,".2%"))

    df123_merge["出库"]=df123_merge["租赁中"]#df123_merge["待发货"]+df123_merge["待收货"]+
    df123_merge["进件出库率"]=df123_merge["出库"]/df123_merge["进件"]
    df123_merge["进件出库率"]=df123_merge["进件出库率"].apply(lambda x:format(x,".2%"))

    df123_merge["订单出库率"]=df123_merge["出库"]/df123_merge["去重订单数"]
    df123_merge["订单出库率"]=df123_merge["订单出库率"].apply(lambda x:format(x,".2%"))
    return df123_merge


# 自动扣款

In [83]:
df_js = df[(df['是否进件']==1)&(df['机审强拒']!=1)]

In [84]:
df_mrs = df_js[df_js["decision_result"].str.contains(pat="免人审",regex=False)==True]
df_mrs_k = df_mrs[df_mrs['auto_pay_tag']==1]
df_mrs_k_count = df_mrs_k['order_number'].count()
df_mrs_q = df_mrs_k['客户取消'].sum()
df_mrs_r = df_mrs_k['人审拒绝'].sum()
df_mrs_c = df_mrs_k['出库前风控强拒'].sum()
df_mrs_ck = df_mrs_k['是否出库'].sum()
df_mrs_k.shape

(125, 64)

In [85]:
df_mrs_ks = df_mrs_k[df_mrs_k['是否出库']==1]
# df_mrs_ks = df_mrs_ks[~df_mrs_ks['cancel_reason'].isna()]
# df_mrs_ks[['order_number', '是否出库', 'cancel_reason']]
df_mrs_ks.shape

(76, 64)

In [86]:
sql2 = '''
SELECT 
om.order_number, tos.money, tos.real_pay_money, tos.sort, tos.refund_date, tos.reality_refund_date, top.create_time as create_time_k
from db_digua_business.t_order  om 
left join db_digua_business.t_order_stages tos on om.id=tos.order_id
left join db_digua_business.t_order_personnel top on om.id=top.order_id
where om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -7 day )              -- 近15天数据
-- AND om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -1 day )
'''
df_order2 = query(sql2)

df2 = df_order2.copy()
df2.shape

(408707, 7)

In [87]:
df3 = pd.merge(df_mrs_k, df2, on='order_number', how='left')
df_order_sum = df3.groupby('order_number').agg({'real_pay_money': 'sum'})
df_order_sum.rename(columns={'real_pay_money': 'pay_money'}, inplace=True)
df4 = pd.merge(df3, df_order_sum, on='order_number', how='left')

df4_success = df4[df4['pay_money']>1]
df4_success_count = df4_success['order_number'].drop_duplicates().count()
df4_success_count # 成功订单
df4_fail = df4[df4['pay_money']<=1]
df4_fail_count = df4_fail['order_number'].drop_duplicates().count()
df4_fail_count # 失败订单
df4_fail_1 =  df4_fail[df4_fail['sort']==1][df4_fail[df4_fail['sort']==1]['money']==1]
df4_fail_1_count = df4_fail_1['order_number'].drop_duplicates().count()
df4_fail_1_count # 首期一元只扣到第一期订单
df4_fail_f1 =  df4_fail[df4_fail['sort']==1][df4_fail[df4_fail['sort']==1]['money']!=1]
df4_fail_f1_count = df4_fail_f1['order_number'].drop_duplicates().count()
df4_fail_f1_count # 首期一元只扣到第一期订单
df4_success_reason = df4_success[['create_time', 'order_number', 'status2',  'cancel_reason', 'rejected', '是否出库']].drop_duplicates()
df4_success_reason

df4_success_reason['取消原因'] = df4_success_reason[df4_success_reason['status2']=='订单取消']['cancel_reason'].str.split('：')
# df4_success_reason['取消原因'] = df4_success_reason['取消原因'].apply(lambda x: x[-1] if x is not None else x)
df4_success_reason_group = df4_success_reason.groupby('取消原因').agg({'order_number': 'count'})
df4_success_reason_group2 =  df4_success_reason.groupby('rejected').agg({'order_number': 'count'})
df4_success_reason_groups = pd.concat([df4_success_reason_group, df4_success_reason_group2])
df4_success_reason_groups['扣款成功订单'] = df4_success_count
# df4_fail_reason_group 

In [88]:
df4_success_reason[df4_success_reason['status2']=='订单取消']['cancel_reason'].str.split('：')

Series([], Name: cancel_reason, dtype: object)

In [89]:
df_mrs_ks[~df_mrs_ks['order_number'].isin(df4_success['order_number'].to_list())]


,create_date,create_time,order_id,order_number,all_money,status,status2,channel_type,sku_attributes,product_name,...,前置拦截,机审强拒,人审拒绝,客户取消,出库前风控强拒,无法联系,是否进件,是否出库,进件前取消,create_hour


In [90]:
# df4_success[~df4_success['cancel_reason'].isna()][['create_time', 'order_number', 'cancel_reason', 'rejected', '是否出库']].drop_duplicates()
df4_success_reason[df4_success_reason['rejected']!='[]']

,create_time,order_number,status2,cancel_reason,rejected,是否出库,取消原因
2376,2024-11-05 16:24:28,A20241105162428127,租赁中,None,[命中云商分低于515强拒(芝麻综合风险等级0豁免)],1,NaN


In [91]:
df4_ck = df4_success[df4_success['是否出库']==1]
# df4_success[['order_number', 'refund_date', 'reality_refund_date']].to_excel('F:/sdfsdf.xlsx')
# df4_success[df4_success['refund_date']<df4_success['reality_refund_date'].dt.date].shape
# df4_ck.shape

In [92]:
# df3[['order_number', 'refund_date', 'reality_refund_date']].to_excel('F:/dfs.xlsx')

In [93]:
# df4_success_reason[df4_success_reason['rejected']!='[]'].drop_duplicates()

In [94]:
df4_success_reason_groups['比例'] = df4_success_reason_groups['order_number'] / df4_success_reason_groups['扣款成功订单']
df4_success_reason_groups['比例'] = df4_success_reason_groups['比例'].apply(lambda x: format(x, '.2%'))
df4_success_reason_groups.rename(columns={'order_number': '取消、拒绝原因数量'}, inplace=True)
df4_success_reason_groups = df4_success_reason_groups.iloc[1:]

In [95]:
df4_success[['order_number', 'sort', 'refund_date', 'reality_refund_date']]
df4_success_1 = df4_success[df4_success['sort']==1][['order_number', 'create_time', 'reality_refund_date']]
df4_success_1.rename(columns={'reality_refund_date': 'sort1_date'}, inplace=True)
df4_success_12 = df4_success[df4_success['sort']==12][['order_number', 'reality_refund_date']]
df4_success_12.rename(columns={'reality_refund_date': 'sort12_date'}, inplace=True)
df4_success_112 = pd.merge(df4_success_1, df4_success_12, on='order_number', how='left').drop_duplicates()
# df4_success_112['date_diff'] = df4_success_112['sort1_date']-df4_success_112['sort12_date']
# df4_success_112['with_15'] = df4_success_112['date_diff'] <= pd.Timedelta(minutes=15)
# df4_success_112['with_15'].value_counts()


# df4_success_112['order_number2'] = np.where(df4_success_112['sort12_date'].isna(), df4_success_112[(df4_success_112['sort1_date']-df4_success_112['create_time'])<=pd.Timedelta(minutes=15)]['order_number'].drop_duplicates(), 
#             df4_success_112[(df4_success_112['sort12_date']-df4_success_112['sort1_date'])<=pd.Timedelta(minutes=15)]['order_number'].drop_duplicates())
# df4_success_112['order_number2']
df4_success_112_na = df4_success_112[df4_success_112['sort12_date'].isna()]
df4_success_112_fna = df4_success_112[~df4_success_112['sort12_date'].isna()]
df4_success_112_count1 = df4_success_112_na[(df4_success_112_na['sort1_date']-df4_success_112_na['create_time'])<=pd.Timedelta(minutes=5)]['order_number'].drop_duplicates().count()
df4_success_112_count2 = df4_success_112_na[(df4_success_112_na['sort1_date']-df4_success_112_na['create_time'])>pd.Timedelta(minutes=5)]['order_number'].drop_duplicates().count()
df4_success_112_count3 = df4_success_112_fna[(df4_success_112_fna['sort12_date']-df4_success_112_fna['sort1_date'])<=pd.Timedelta(minutes=5)]['order_number'].drop_duplicates().count()
df4_success_112_count4 = df4_success_112_fna[(df4_success_112_fna['sort12_date']-df4_success_112_fna['sort1_date'])>pd.Timedelta(minutes=5)]['order_number'].drop_duplicates().count()

In [96]:
df4_success_112_count4

np.int64(7)

In [97]:
df4_success1 = df4_success[['order_number', '客户取消', '人审拒绝', '出库前风控强拒','待审核', '是否出库']].drop_duplicates()

In [98]:
df_auto = pd.DataFrame({
    '自动扣款订单': [df_mrs_k_count],
    '自动扣款成功订单': [df4_success_count-df4_success_112_count1-df4_success_112_count3],
    '人审拒绝': [df_mrs_r],
    '客户取消': [df_mrs_q],
    '出库前风控强拒': [df_mrs_c],
    '出库': [df_mrs_ck],
    '自动扣款失败订单': [df4_fail_count+df4_success_112_count2+df4_success_112_count4],
    '自动扣款失败订单（首期一元）': [df4_fail_1_count],
    '自动扣款失败订单（非首期一元）': [df4_fail_f1_count]
})
df_auto['成功比例'] = df_auto['自动扣款成功订单'] / df_auto['自动扣款订单']
df_auto['成功比例'] = df_auto['成功比例'].apply(lambda x: format(x, '.2%'))

df_auto['人审拒绝比例'] = df_auto['人审拒绝'] / df_auto['自动扣款订单']
df_auto['人审拒绝比例'] = df_auto['人审拒绝比例'].apply(lambda x: format(x, '.2%'))

df_auto['客户取消比例'] = df_auto['客户取消'] / df_auto['自动扣款订单']
df_auto['客户取消比例'] = df_auto['客户取消比例'].apply(lambda x: format(x, '.2%'))

df_auto['出库前风控强拒比例'] = df_auto['出库前风控强拒'] / df_auto['自动扣款订单']
df_auto['出库前风控强拒比例'] = df_auto['出库前风控强拒比例'].apply(lambda x: format(x, '.2%'))

df_auto['出库比例'] = df_auto['出库'] / df_auto['自动扣款订单']
df_auto['出库比例'] = df_auto['出库比例'].apply(lambda x: format(x, '.2%'))

df_auto['失败比例'] = df_auto['自动扣款失败订单'] / df_auto['自动扣款订单']
df_auto['失败比例'] = df_auto['失败比例'].apply(lambda x: format(x, '.2%'))

df_auto['失败比例（首期一元）'] = df_auto['自动扣款失败订单（首期一元）'] / df_auto['自动扣款订单']
df_auto['失败比例（首期一元）'] = df_auto['失败比例（首期一元）'].apply(lambda x: format(x, '.2%'))

df_auto['失败比例（非首期一元）'] = df_auto['自动扣款失败订单（非首期一元）'] / df_auto['自动扣款订单']
df_auto['失败比例（非首期一元）'] = df_auto['失败比例（非首期一元）'].apply(lambda x: format(x, '.2%'))

df_auto = df_auto[['自动扣款订单','自动扣款成功订单','成功比例','人审拒绝','人审拒绝比例','客户取消','客户取消比例','出库前风控强拒','出库前风控强拒比例','出库',
                    '出库比例','自动扣款失败订单','失败比例','自动扣款失败订单（首期一元）','失败比例（首期一元）','自动扣款失败订单（非首期一元）','失败比例（非首期一元）']]
df_auto

,自动扣款订单,自动扣款成功订单,成功比例,人审拒绝,人审拒绝比例,客户取消,客户取消比例,出库前风控强拒,出库前风控强拒比例,出库,出库比例,自动扣款失败订单,失败比例,自动扣款失败订单（首期一元）,失败比例（首期一元）,自动扣款失败订单（非首期一元）,失败比例（非首期一元）
0,125,7,5.60%,18,14.40%,19,15.20%,6,4.80%,76,60.80%,54,43.20%,19,15.20%,28,22.40%


In [99]:

df4_success_q = df4_success1['客户取消'].sum()
df4_success_r = df4_success1['人审拒绝'].sum()
df4_success_c = df4_success1['出库前风控强拒'].sum()
df4_success_d = df4_success1['待审核'].sum()
df4_success_ck = df4_success1['是否出库'].sum()

# df4_success[df4_success['reality_refund_date']]

df4_success_group = pd.DataFrame({
    '扣款成功订单': [df4_success_count],
    '自动扣款成功订单': [df4_success_112_count1+df4_success_112_count3],
    '自动扣款挽留订单': [df4_success_112_count2+df4_success_112_count4],
    '人审拒绝': [df4_success_r],
    '客户取消': [df4_success_q],
    '出库前风控强拒': [df4_success_c],
    '待审核': [df4_success_d],
    '出库': [df4_success_ck]
})

df4_success_group['自动扣款成功订单比例'] = df4_success_group['自动扣款成功订单'] / df4_success_group['扣款成功订单']
df4_success_group['自动扣款成功订单比例'] = df4_success_group['自动扣款成功订单比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['自动扣款挽留订单比例'] = df4_success_group['自动扣款挽留订单'] / df4_success_group['扣款成功订单']
df4_success_group['自动扣款挽留订单比例'] = df4_success_group['自动扣款挽留订单比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['人审拒绝比例'] = df4_success_group['人审拒绝'] / df4_success_group['扣款成功订单']
df4_success_group['人审拒绝比例'] = df4_success_group['人审拒绝比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['客户取消比例'] = df4_success_group['客户取消'] / df4_success_group['扣款成功订单']
df4_success_group['客户取消比例'] = df4_success_group['客户取消比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['出库前风控强拒比例'] = df4_success_group['出库前风控强拒'] / df4_success_group['扣款成功订单']
df4_success_group['出库前风控强拒比例'] = df4_success_group['出库前风控强拒比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['待审核比例'] = df4_success_group['待审核'] / df4_success_group['扣款成功订单']
df4_success_group['待审核比例'] = df4_success_group['待审核比例'].apply(lambda x: format(x, '.2%'))

df4_success_group['出库比例'] = df4_success_group['出库'] / df4_success_group['扣款成功订单']
df4_success_group['出库比例'] = df4_success_group['出库比例'].apply(lambda x: format(x, '.2%'))

df4_success_group = df4_success_group[['扣款成功订单', '自动扣款成功订单', '自动扣款成功订单比例', '自动扣款挽留订单', '自动扣款挽留订单比例', '人审拒绝','人审拒绝比例','客户取消','客户取消比例','出库前风控强拒','出库前风控强拒比例', '待审核', '出库',
                    '出库比例']]
df4_success_group

,扣款成功订单,自动扣款成功订单,自动扣款成功订单比例,自动扣款挽留订单,自动扣款挽留订单比例,人审拒绝,人审拒绝比例,客户取消,客户取消比例,出库前风控强拒,出库前风控强拒比例,待审核,出库,出库比例
0,78,71,91.03%,7,8.97%,0,0.00%,0,0.00%,0,0.00%,2,76,97.44%


In [100]:
# with pd.ExcelWriter('F:/需求/七月需求/自动扣款1.xlsx', engine='openpyxl') as writer:
#     df_auto.to_excel(writer, sheet_name='自动扣款汇总', index=False)
#     df4_success_group.to_excel(writer, sheet_name='扣款成功转化', index=False)
    # df4_success_reason_groups.to_excel(writer, sheet_name='取消，拒绝原因汇总')

In [101]:
df4_success_reason_groups

,取消、拒绝原因数量,扣款成功订单,比例
[命中云商分低于515强拒(芝麻综合风险等级0豁免)],1,78,1.28%


# 非自动扣款

In [116]:
df_mrs_fk = df_mrs[df_mrs['auto_pay_tag']!=1]


In [117]:
df_mrs_fk['机型'] = df_mrs_fk['product_name'].str.extract(r'(iPhone15Pro)')[0]
df_mrs_fk['机型'].drop_duplicates()

624     iPhone15Pro
1408            NaN
Name: 机型, dtype: object

## iPhone 15Pro_512G

In [120]:
df_mrs_fk[(df_mrs_fk['机型']=='iPhone15Pro')&(df_mrs_fk['内存']=='512G')].shape

(14, 65)

# 八派|九州

In [124]:
df_mrs_fk[(df_mrs_fk['来源渠道']=='八派信息')|((df_mrs_fk['来源渠道']=='九州信息'))].shape

(43, 65)